## Importing data manipulation libraries 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from matplotlib.image import imread
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D,Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

## Loading dataset

Custom curated dataset includiing 476 images.
Hand-signs are annotated in folder format.

The following pre-processing was applied to each image:
* Auto-orientation of pixel data (with EXIF-orientation stripping)
* Resize to 640x640 (Stretch)

The following augmentation was applied to create 3 versions of each source image:
* Randomly crop between 0 and 25 percent of the image

In [ ]:
train_dir = "/Users/casarulez/Projects/IoT /dataset-annotated/train"
test_dir = "/Users/casarulez/Projects/IoT /dataset-annotated/test"

In [ ]:
def display_samples(directory):
    classes = os.listdir(directory)
    fig, axes = plt.subplots(len(classes), 4, figsize=(16, 16))

    for i, class_name in enumerate(classes):
        if class_name == ".DS_Store":  # Skip .DS_Store directory
            continue
        
        class_dir = os.path.join(directory, class_name)
        img_names = os.listdir(class_dir)[:4]  # Load the first four images from each class
        for j, img_name in enumerate(img_names):
            img_path = os.path.join(class_dir, img_name)
            img = imread(img_path)
            axes[i, j].imshow(img)
            axes[i, j].set_title(class_name)
            axes[i, j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Display sample images from the training dataset
display_samples(train_dir)

In [ ]:
def load_data(directory):
    images = []
    labels = []
    classes = sorted(os.listdir(directory))  # Assuming classes are named as directories
    for i, class_name in enumerate(classes):
        if class_name == '.DS_Store':  # Skip .DS_Store file if present
            continue
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            if filename == '.DS_Store':  # Skip .DS_Store file if present
                continue
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)  # Read image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB (if needed)
            images.append(img)
            labels.append(i-1)  # Assign label based on class index
    return np.array(images), np.array(labels)

In [ ]:
# Load training and testing data
train_images, train_labels = load_data(train_dir)
test_images, test_labels = load_data(test_dir)

In [ ]:
# Display shape of arrays
print("Shape of train images array:", train_images.shape)
print("Shape of train labels array:", train_labels.shape)
print("Shape of test images array:", test_images.shape)
print("Shape of test labels array:", test_labels.shape)

In [ ]:
train_labels

## Defining model architecture  

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load MobileNetV2 pre-trained model without the top classification layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(640, 640, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='gelu')(x)  # Add additional dense layer if needed
predictions = Dense(4, activation='softmax')(x)  # Adjust the number of classes (4 in this example)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model architecture
model.summary()


## Fitting the model 

In [ ]:
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

In [ ]:
train_loss, train_acc = model.evaluate(train_images, train_labels)
print('Train data accuracy: {:.2f}%'.format(train_acc * 100))

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test data accuracy: {:.2f}%'.format(test_acc * 100))

In [ ]:
# Load and preprocess the image
image_path = '/Users/casarulez/Downloads/ls.jpg'  # Corrected file path
image = cv2.imread(image_path)
image = image.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

# Expand dimensions to create a batch of size 1
image = np.expand_dims(image, axis=0)

# Make prediction
predictions = model.predict(image)

# Display the predictions
print(predictions)

In [ ]:
import numpy as np

# Assuming predictions is a NumPy array containing probabilities for each class
# Each row of predictions corresponds to the predicted probabilities for one sample

# Assuming label_names is a list of label names corresponding to the classes
label_names = ['Fan', 'OFF', 'ON', 'light']  # Update with your actual label names

# Predict the labels for the test set
predicted_labels = np.argmax(predictions, axis=1)

# Map the predicted label indices to their corresponding label names
predicted_label_names = [label_names[label_index] for label_index in predicted_labels]

# Display the predicted label names
print("Predicted label names:", predicted_label_names)


## Video capture

In [ ]:
import cv2
import numpy as np

# Function to preprocess a single image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (640, 640))  # Resize the image to match the model input size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Function to overlay class probabilities on the image as text
def overlay_class_probabilities(image, predictions):
    classes = ['Fan', 'OFF', 'ON', 'light']  # Update with your actual class names
    if predictions.any():  # Check if any element in predictions array is nonzero
        probabilities = predictions[0]  # Extract probabilities from the array
        for i, prob in enumerate(probabilities):
            text = f'{classes[i]}: {prob:.2f}'  # Format probability to two decimal places
            cv2.putText(image, text, (10, 30 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    return image
# Load and preprocess the image
image_path = '/Users/casarulez/Downloads/off.jpg'  # Corrected file path
image = cv2.imread(image_path)
image = image.astype('float32') / 255.0  # Normalize the pixel values to [0, 1]

# Expand dimensions to create a batch of size 1
image = np.expand_dims(image, axis=0)

# Make prediction
predictions = model.predict(image)

# Display the predictions
print(predictions)

# Overlay class probabilities on the image
output_image = overlay_class_probabilities(image.copy(), predictions)

# Display the original and output images
cv2.imshow('Original Image', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
cv2.imshow('Output Image', cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()
